## NLP教程

### 1、认识自然语言处理
### 2、掌握文本分词工具

### 3、词提取算法介绍
### 4、文本分类聚类处理
### 5、深度学习的NLP
### 6、一些进化结构的讲解

### 1、认识自然语言处理（处理是为了理解）
自然语言处理 (NLP) 使用机器学习来剖析文本的结构和含义。借助自然语言处理应用，组织可以分析文本并提取关于人物、地点和事件的信息，以更好地理解社交媒体内容的情感和客户对话。


自然语言处理应用用于从非结构化文本数据中发掘洞见，并让您能够访问所提取出的信息，以生成有关这些数据的新的理解。您可以使用 Python、TensorFlow 和 PyTorch 构建自然语言处理示例。

![title](NLP用途.png)

#### 这里有一个小问题：我们再做数据挖掘的时候和图像识别的时候，实际上我们是在对什么进行操作？数字（或者说矩阵）！是不是？那么怎么样才能让文本变成数字呢？

## 2、掌握文本分词工具jieba https://github.com/fxsjy/jieba

In [1]:
! pip install jieba#安装jieba

zsh:1: /usr/local/bin/pip: bad interpreter: /System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: no such file or directory
ERROR: Invalid requirement: 'jieba#安装jieba'


### 2.1 分词

支持四种分词模式：

精确模式，试图将句子最精确地切开，适合文本分析；

全模式，把句子中所有的可以成词的词语都扫描出来, 速度非常快，但是不能解决歧义；

搜索引擎模式，在精确模式的基础上，对长词再次切分，提高召回率，适合用于搜索引擎分词。

paddle模式，利用PaddlePaddle深度学习框架，训练序列标注（双向GRU）网络模型实现分词。同时支持词性标注。paddle模式使用需安装paddlepaddle-tiny，pip install paddlepaddle-tiny==1.6.1。目前paddle模式支持jieba v0.40及以上版本。jieba v0.40以下版本，请升级jieba，pip install jieba --upgrade 。PaddlePaddle官网

In [2]:
! pip install paddlepaddle

zsh:1: /usr/local/bin/pip: bad interpreter: /System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: no such file or directory


In [3]:
import paddle
paddle.enable_static()

In [4]:
import jieba

jieba.enable_paddle()# 启动paddle模式。 0.40版之后开始支持，早期版本不支持
strs=["我来到北京清华大学","乒乓球拍卖完了","中国科学技术大学"]
for str in strs:
    seg_list = jieba.cut(str,use_paddle=True) # 使用paddle模式
    print("Paddle Mode: " + '/'.join(list(seg_list)))

seg_list = jieba.cut("我来到北京清华大学", cut_all=True)
print("Full Mode: " + "/ ".join(seg_list))  # 全模式

seg_list = jieba.cut("我来到北京清华大学", cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))  # 精确模式

seg_list = jieba.cut("他来到了网易杭研大厦")  # 默认是精确模式
print(", ".join(seg_list))

seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造")  # 搜索引擎模式
print(", ".join(seg_list))

Paddle enabled successfully......
Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/3w/772jr2311zsbz371z3v36rdw0000gn/T/jieba.cache


Paddle Mode: 我/来到/北京清华大学
Paddle Mode: 乒乓球/拍卖/完/了
Paddle Mode: 中国科学技术大学


Loading model cost 0.994 seconds.
Prefix dict has been built successfully.


Full Mode: 我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学
Default Mode: 我/ 来到/ 北京/ 清华大学
他, 来到, 了, 网易, 杭研, 大厦
小明, 硕士, 毕业, 于, 中国, 科学, 学院, 科学院, 中国科学院, 计算, 计算所, ，, 后, 在, 日本, 京都, 大学, 日本京都大学, 深造


### 2.2用户词典-用来解决某些特定词的。

In [18]:
from __future__ import print_function, unicode_literals
import sys
sys.path.append("../")
import jieba
jieba.load_userdict("userdict.txt")
import jieba.posseg as pseg

jieba.add_word('石墨烯')
jieba.add_word('凱特琳')
jieba.del_word('自定义词')
jieba.del_word("李小福")


test_sent = (
"李小福是创新办主任也是云计算方面的专家; 什么是八一双鹿\n"
"例如我输入一个带“韩玉赏鉴”的标题，在自定义词库中也增加了此词为N类\n"
"「台中」正確應該不會被切開。mac上可分出「石墨烯」；此時又可以分出來凱特琳了。"
)
seg_list1 = jieba.cut("李小福是创新办主任也是云计算方面的专家; 什么是八一双鹿\n")
print("Full Mode: " + "/ ".join(seg_list1)) 


Full Mode: 李/ 小/ 福/ 是/ 创新办/ 主任/ 也/ 是/ 云计算/ 方面/ 的/ 专家/ ;/  / 什么/ 是/ 八一双鹿/ 



### 2.3.1、关键词的提取 （基于 TF-IDF 算法的关键词抽取）

TF-IDF(Term Frequency-Inverse Document Frequency, 词频-逆文件频率)是一种用于资讯检索与资讯探勘的常用加权技术。TF-IDF是一种统计方法，用以评估一字词对于一个文件集或一个语料库中的其中一份文件的重要程度。字词的重要性随着它在文件中出现的次数成正比增加，但同时会随着它在语料库中出现的频率成反比下降。

上述引用总结就是, 一个词语在一篇文章中出现次数越多, 同时在所有文档中出现次数越少, 越能够代表该文章。这也就是TF-IDF的含义。



In [21]:
import jieba
import jieba.analyse
content = open("wenben.txt", 'rb').read()

tags = jieba.analyse.extract_tags(content, topK=10,withWeight=False,allowPOS=('ns', 'n', 'vn', 'v'))
"""sentence, topK=20, withWeight=False, allowPOS=()
sentence 为待提取的文本
topK 为返回几个 TF/IDF 权重最大的关键词，默认值为 20
withWeight 为是否一并返回关键词权重值，默认值为 False
allowPOS 仅包括指定词性的词，默认值为空，即不筛选
"""
print(",".join(tags))

俄罗斯,武器,北约,弹药,提供,报道,玩火,军事援助,照会,外长


### 2.3.2、关键词的提取 （基于 Text-Rank 算法的关键词抽取）
TextRank 算法是一种用于文本的基于图的排序算法，通过把文本分割成若干组成单元（句子），构建节点连接图，用句子之间的相似度作为边的权重，通过循环迭代计算句子的TextRank值，最后抽取排名高的句子组合成文本摘要。本文介绍了抽取型文本摘要算法TextRank，并使用Python实现TextRank算法在多篇单领域文本数据中抽取句子组成摘要的应用。

In [7]:
tags1 = jieba.analyse.textrank(content, topK=10, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v'))
print(",".join(tags1))

俄罗斯,武器,弹药,提供,国家,系统,火箭,北约,美国,海马


##### 去停用词

In [26]:
content = open("wenben.txt", 'rb').read()

jieba.analyse.set_stop_words("stopwords.txt")

tags = jieba.analyse.extract_tags(content, topK=10)

print(",".join(tags))

武器,北约,弹药,提供,拉夫罗夫,8.2,这轮,NASAMS,15,155


###  2.4词性标注

In [9]:
import jieba
import jieba.posseg as pseg
words = pseg.cut("我爱北京天安门") #jieba默认模式
jieba.enable_paddle() #启动paddle模式。
words = pseg.cut("我爱北京天安门",use_paddle=True) #paddle模式
for word, flag in words:
   print('%s %s' % (word, flag))


Paddle enabled successfully......


我 r
爱 v
北京 LOC
天安门 LOC


## 3、词提取算法介绍

### 3.1、TF-IDF算法的介绍

TF(词频)-IDF（逆文档频率）
 
TF = 某个词在文章中出现的次数/文章中的总词数

IDF = log（语料库中的文档总数/（包含该词的文档数+1））

然后再就是按照降序排列，取前面的几个数。

In [10]:
corpus = ['通过训练得到的词向量我们能够进行对应的自然语言处理工作。比方求相似词、关键词聚类等。',
        '自然语言处理的课程很好呀，大家快点收藏，点赞！',
        '波老师讲的这个自然语言处理课程真的很nice',
        '原来自然语言处理课程学要这样子学呀？']

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer()
tfidf_matrix = tfidf_vec.fit_transform(corpus)
# 得到语料库所有不重复的词
print(tfidf_vec.get_feature_names())
# 得到每个单词对应的id值
print(tfidf_vec.vocabulary_)
# 得到每个句子所对应的向量，向量里数字的顺序是按照词语的id顺序来的
print(tfidf_matrix.toarray())

['关键词聚类等', '原来自然语言处理课程学要这样子学呀', '大家快点收藏', '比方求相似词', '波老师讲的这个自然语言处理课程真的很nice', '点赞', '自然语言处理的课程很好呀', '通过训练得到的词向量我们能够进行对应的自然语言处理工作']
{'通过训练得到的词向量我们能够进行对应的自然语言处理工作': 7, '比方求相似词': 3, '关键词聚类等': 0, '自然语言处理的课程很好呀': 6, '大家快点收藏': 2, '点赞': 5, '波老师讲的这个自然语言处理课程真的很nice': 4, '原来自然语言处理课程学要这样子学呀': 1}
[[0.57735027 0.         0.         0.57735027 0.         0.
  0.         0.57735027]
 [0.         0.         0.57735027 0.         0.         0.57735027
  0.57735027 0.        ]
 [0.         0.         0.         0.         1.         0.
  0.         0.        ]
 [0.         1.         0.         0.         0.         0.
  0.         0.        ]]
